In [1]:
pip install stanza

     |████████████████████████████████| 235kB 2.8MB/s 


In [2]:
import stanza

In [3]:
stanza.download('grc')

2020-11-10 22:55:19 INFO: Downloading default packages for language: grc (Ancient_Greek)...
2020-11-10 22:55:39 INFO: Finished downloading models and saved to /root/stanza_resources.


In [4]:
import pandas as pd 
import gensim
from gensim import corpora,models
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import nltk
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer #tokenizes and counts words
from sklearn.feature_extraction.text import TfidfVectorizer #alternatively, use TfidfTransformer()
from sklearn import metrics
import requests
from sklearn.feature_selection import RFE, chi2
from scipy.spatial import distance
from pandas.core.frame import DataFrame
from nltk import FreqDist
from sklearn.metrics.pairwise import cosine_similarity
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import string
from scipy.stats import entropy
from numpy.linalg import norm
from scipy.spatial import distance
import spacy
from collections import Counter

In [19]:
from spacy.lang.el import Greek

In [7]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)


Mounted at /content/drive


['Getting started.pdf',
 'image (1).jpg',
 'image.jpg',
 'C347414E-901F-4ADD-A460-3B99C03799B4.png',
 'Introduction to Scientific Programming and Simulation Using R.pdf',
 'RDataScience(1).pdf',
 'database group project.gdoc',
 'Database Data.gsheet',
 'Text Analytics Phase 1.gslides',
 '2018GMAT课件',
 'Untitled presentation (1).gslides',
 '9526f2636c319b9ed60bdc9c605f7725.mp4',
 'Camera Roll.zip',
 'WIN_20200323_11_22_38_Pro.mp4',
 'Untitled spreadsheet.gsheet',
 'Big Data Analytics Proposal.gslides',
 'Discover the correlation between Public transportation facilities investment and the frequency of Traffic accident in United States.gdoc',
 'Untitled presentation.gslides',
 'gmat.exe',
 'Web Analytics.zip',
 'Data Mining.zip',
 'datamodeler-19.2.0.182.1216-x64.zip',
 'Bootcamp 课件.zip',
 'Torrey (1911).pdf',
 'Torrey (1911).gdoc',
 'Bootcamp 课件 (1).zip',
 'Bible_Data.csv',
 'OtherApostlesWork_Data.csv',
 'Colab Notebooks',
 'drive',
 'Peter word_count.jpg',
 'John word_count.jpg',
 'Jud

In [8]:
df = pd.read_csv('NA28_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')
df2 = pd.read_csv('SBL_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')
df3 = pd.read_csv('Tyndale_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')

In [9]:
def cosine_text(text):
    text_list = []
    for t in text['Verse']:
        text_list.append(t)

    text_str = ' '.join(text_list)
    return text_str

In [10]:
stanza.download('el')

2020-11-10 23:15:51 INFO: Downloading default packages for language: el (Greek)...
2020-11-10 23:16:58 INFO: Finished downloading models and saved to /root/stanza_resources.


In [11]:
nlp = stanza.Pipeline(lang='el', processors='tokenize,pos,mwt,lemma', tokenize_pretokenized=True)

2020-11-10 23:17:17 INFO: Loading these models for language: el (Greek):
| Processor | Package |
-----------------------
| tokenize  | gdt     |
| mwt       | gdt     |
| pos       | gdt     |
| lemma     | gdt     |

2020-11-10 23:17:17 INFO: Use device: cpu
2020-11-10 23:17:17 INFO: Loading: tokenize
2020-11-10 23:17:17 INFO: Loading: mwt
2020-11-10 23:17:17 INFO: Loading: pos
2020-11-10 23:17:18 INFO: Loading: lemma
2020-11-10 23:17:18 INFO: Done loading processors!


In [12]:
def word_list(text):
    spacy_stopwords = spacy.lang.el.stop_words.STOP_WORDS
    # spacy_stopwords is a hardcoded set
    tokens = nlp(text).get('lemma')
    tokens1 = nlp(hbr).get('lemma')
    #token_list = []
    lemma_list = []
    lemma_list1 = []
    for token in tokens:
        if token.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list.append(token)
    for token in tokens1:
        if token.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list1.append(token)
    #token_list
    dict1 = {}
    dict2 = {}
    for key in lemma_list:
        dict1[key] = dict1.get(key, 0) + 1
    for key in lemma_list1:
        dict2[key] = dict2.get(key, 0) + 1
    l1 = []
    l2 = []
    for word in dict1:
        if word not in dict2:
            l2.append(word)
    for word in dict2:
        if word not in dict1:
            l1.append(word)
    for x in l1:
        dict1[x] = dict1.get(x,0)
    for y in l2:
        dict2[y] = dict2.get(y,0)
    s_dict1 = sorted(dict1.items())
    s_dict2 = sorted(dict2.items())
    m1 = []
    m2 = []
    for m in s_dict1:
        m1.append(m[1])
    for n in s_dict2:
        m2.append(n[1])
    return [m1,m2]

In [13]:
def dice(im1, im2):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum()) 

In [14]:
def word_list1(text):
    spacy_stopwords = spacy.lang.el.stop_words.STOP_WORDS
    # spacy_stopwords is a hardcoded set
    tokens = nlp(text).get('lemma')
    tokens1 = nlp(pl_s1).get('lemma')
    #token_list = []
    lemma_list = []
    lemma_list1 = []
    for token in tokens:
        if token.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list.append(token)
    for token in tokens1:
        if token.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list1.append(token)
    #token_list
    dict1 = {}
    dict2 = {}
    for key in lemma_list:
        dict1[key] = dict1.get(key, 0) + 1
    for key in lemma_list1:
        dict2[key] = dict2.get(key, 0) + 1
    l1 = []
    l2 = []
    for word in dict1:
        if word not in dict2:
            l2.append(word)
    for word in dict2:
        if word not in dict1:
            l1.append(word)
    for x in l1:
        dict1[x] = dict1.get(x,0)
    for y in l2:
        dict2[y] = dict2.get(y,0)
    s_dict1 = sorted(dict1.items())
    s_dict2 = sorted(dict2.items())
    m1 = []
    m2 = []
    for m in s_dict1:
        m1.append(m[1])
    for n in s_dict2:
        m2.append(n[1])
    return [m1,m2]

## NA28_version

In [15]:
Matthew = df[df.BookName == 'Mt']
Matthew['Author'] = 'Matthew'
Mark = df[df.BookName == 'Mk']
Mark['Author'] = 'Mark'
Luke = df[df.BookName == 'Lk']
Luke['Author'] = 'Luke'
John = df[df.BookName == 'Joh']
John['Author'] = 'John'
Acts = df[df.BookName == 'Apg']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df[df.BookName == 'Jak']
James['Author'] = 'James'
f_Peter = df[df.BookName == '1.Petr']
s_Peter = df[df.BookName == '2.Petr']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df[df.BookName == '1.Joh']
s_John = df[df.BookName == '2.Joh']
t_John = df[df.BookName == '3.Joh']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df[df.BookName == 'Jud']
Jude['Author'] = 'Jude'
Rev = df[df.BookName == 'Offb']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [16]:
Romans = df[df.BookName == 'Röm']
Romans['Author'] = 'Paul'
f_cor = df[df.BookName == '1.Kor']
f_cor['Author'] = 'Paul'
s_cor = df[df.BookName == '2.Kor']
s_cor['Author'] = 'Paul'
Gal = df[df.BookName == 'Gal']
Gal['Author'] = 'Paul'
Eph = df[df.BookName == 'Eph']
Eph['Author'] = 'Paul'
Phi = df[df.BookName == 'Phil']
Phi['Author'] = 'Paul'
Col = df[df.BookName == 'Kol']
Col['Author'] = 'Paul'
f_the = df[df.BookName == '1.Thess']
f_the['Author'] = 'Paul'
s_the = df[df.BookName == '2.Thess']
s_the['Author'] = 'Paul'
f_tim = df[df.BookName == '1.Tim']
f_tim['Author'] = 'Paul'
s_tim = df[df.BookName == '2.Tim']
s_tim['Author'] = 'Paul'
Titus = df[df.BookName == 'Tit']
Titus['Author'] = 'Paul'
Philemon = df[df.BookName == 'Phlm']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df[df.BookName == 'heb']
Heb['Author'] = 'Paul?'

In [17]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [20]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [23]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
s_cosine = []
for x in range(0,len(documents)):
    
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine.append(cos[1])
    

In [26]:
NA28 = {'StanzaCosine':s_cosine, 'StanzaDice':s_dice, 'StanzaHellinger':s_hel, 'StanzaJaccard':s_jac, 'StanzaJSdivergence':s_jen}
NA28 = pd.DataFrame(NA28)

In [27]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [28]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [29]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
s_cosine1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine1.append(cos[1])

In [30]:
NA281 = {'StanzaCosine':s_cosine1, 'StanzaDice':s_dice1, 'StanzaHellinger':s_hel1, 'StanzaJaccard':s_jac1, 'StanzaJSdivergence':s_jen1}
NA281 = pd.DataFrame(NA281)

In [31]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [32]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
s_cosine2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine2.append(cos[1])

In [33]:
NA282 = {'StanzaCosine':s_cosine2, 'StanzaDice':s_dice2, 'StanzaHellinger':s_hel2, 'StanzaJaccard':s_jac2, 'StanzaJSdivergence':s_jen2}
NA282 = pd.DataFrame(NA282)

In [34]:
NA28_all = pd.concat([NA28, NA281, NA282], axis = 0)

## SBL_version

In [35]:
Matthew = df2[df2.BookName == 'Mt']
Matthew['Author'] = 'Matthew'
Mark = df2[df2.BookName == 'Mk']
Mark['Author'] = 'Mark'
Luke = df2[df2.BookName == 'Lk']
Luke['Author'] = 'Luke'
John = df2[df2.BookName == 'Jn']
John['Author'] = 'John'
Acts = df2[df2.BookName == 'Ac']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df2[df2.BookName == 'Jas']
James['Author'] = 'James'
f_Peter = df2[df2.BookName == '1Pe']
s_Peter = df2[df2.BookName == '2Pe']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df2[df2.BookName == '1Jn']
s_John = df2[df2.BookName == '2Jn']
t_John = df2[df2.BookName == '3Jn']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df2[df2.BookName == 'Jud']
Jude['Author'] = 'Jude'
Rev = df2[df2.BookName == 'Re']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [36]:
Romans = df2[df2.BookName == 'Ro']
Romans['Author'] = 'Paul'
f_cor = df2[df2.BookName == '1Co']
f_cor['Author'] = 'Paul'
s_cor = df2[df2.BookName == '2Co']
s_cor['Author'] = 'Paul'
Gal = df2[df2.BookName == 'Ga']
Gal['Author'] = 'Paul'
Eph = df2[df2.BookName == 'Eph']
Eph['Author'] = 'Paul'
Phi = df2[df2.BookName == 'Php']
Phi['Author'] = 'Paul'
Col = df2[df2.BookName == 'Col']
Col['Author'] = 'Paul'
f_the = df2[df2.BookName == '1Th']
f_the['Author'] = 'Paul'
s_the = df2[df2.BookName == '2Th']
s_the['Author'] = 'Paul'
f_tim = df2[df2.BookName == '1Ti']
f_tim['Author'] = 'Paul'
s_tim = df2[df2.BookName == '2Ti']
s_tim['Author'] = 'Paul'
Titus = df2[df2.BookName == 'Tit']
Titus['Author'] = 'Paul'
Philemon = df2[df2.BookName == 'Phm']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df2[df2.BookName == 'Heb']
Heb['Author'] = 'Paul?'

In [37]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [38]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [39]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
s_cosine = []
for x in range(0,len(documents)):
    
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine.append(cos[1])
    

In [40]:
SBL = {'StanzaCosine':s_cosine, 'StanzaDice':s_dice, 'StanzaHellinger':s_hel, 'StanzaJaccard':s_jac, 'StanzaJSdivergence':s_jen}
SBL = pd.DataFrame(SBL)

In [41]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [42]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [43]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
s_cosine1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine1.append(cos[1])

In [44]:
SBL1 = {'StanzaCosine':s_cosine1, 'StanzaDice':s_dice1, 'StanzaHellinger':s_hel1, 'StanzaJaccard':s_jac1, 'StanzaJSdivergence':s_jen1}
SBL1 = pd.DataFrame(SBL1)

In [45]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [46]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
s_cosine2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine2.append(cos[1])

In [47]:
SBL2 = {'StanzaCosine':s_cosine2, 'StanzaDice':s_dice2, 'StanzaHellinger':s_hel2, 'StanzaJaccard':s_jac2, 'StanzaJSdivergence':s_jen2}
SBL2 = pd.DataFrame(SBL2)

In [48]:
SBL_all = pd.concat([SBL, SBL1, SBL2], axis = 0)

## Tyndale_version

In [49]:
Matthew = df3[df3.BookName == 'Matthew']
Matthew['Author'] = 'Matthew'
Mark = df3[df3.BookName == 'Mark']
Mark['Author'] = 'Mark'
Luke = df3[df3.BookName == 'Luke']
Luke['Author'] = 'Luke'
John = df3[df3.BookName == 'John']
John['Author'] = 'John'
Acts = df3[df3.BookName == 'Acts']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df3[df3.BookName == 'James']
James['Author'] = 'James'
f_Peter = df3[df3.BookName == '1Peter']
s_Peter = df3[df3.BookName == '2Peter']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df3[df3.BookName == '1John']
s_John = df3[df3.BookName == '2John']
t_John = df3[df3.BookName == '3John']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df3[df3.BookName == 'Jude']
Jude['Author'] = 'Jude'
Rev = df3[df3.BookName == 'Revelation']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [50]:
Romans = df3[df3.BookName == 'Romans']
Romans['Author'] = 'Paul'
f_cor = df3[df3.BookName == '1Corinthians']
f_cor['Author'] = 'Paul'
s_cor = df3[df3.BookName == '2Corinthians']
s_cor['Author'] = 'Paul'
Gal = df3[df3.BookName == 'Galatians']
Gal['Author'] = 'Paul'
Eph = df3[df3.BookName == 'Ephesians']
Eph['Author'] = 'Paul'
Phi = df3[df3.BookName == 'Philippians']
Phi['Author'] = 'Paul'
Col = df3[df3.BookName == 'Colossians']
Col['Author'] = 'Paul'
f_the = df3[df3.BookName == '1Thessalonians']
f_the['Author'] = 'Paul'
s_the = df3[df3.BookName == '2Thessalonians']
s_the['Author'] = 'Paul'
f_tim = df3[df3.BookName == '1Timothy']
f_tim['Author'] = 'Paul'
s_tim = df3[df3.BookName == '2Timothy']
s_tim['Author'] = 'Paul'
Titus = df3[df3.BookName == 'Titus']
Titus['Author'] = 'Paul'
Philemon = df3[df3.BookName == 'Philemon']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df3[df3.BookName == 'Hebrews']
Heb['Author'] = 'Paul?'

In [51]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [52]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [53]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
s_cosine = []
for x in range(0,len(documents)):
    
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine.append(cos[1])
    

In [54]:
Tyndale = {'StanzaCosine':s_cosine, 'StanzaDice':s_dice, 'StanzaHellinger':s_hel, 'StanzaJaccard':s_jac, 'StanzaJSdivergence':s_jen}
Tyndale = pd.DataFrame(Tyndale)

In [55]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [56]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [57]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
s_cosine1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine1.append(cos[1])

In [58]:
Tyndale1 = {'StanzaCosine':s_cosine1, 'StanzaDice':s_dice1, 'StanzaHellinger':s_hel1, 'StanzaJaccard':s_jac1, 'StanzaJSdivergence':s_jen1}
Tyndale1 = pd.DataFrame(Tyndale1)

In [59]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [60]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
s_cosine2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))
    cos = tuple(cosine_similarity(documents[x])[0])
    s_cosine2.append(cos[1])

In [62]:
Tyndale2 = {'StanzaCosine':s_cosine2, 'StanzaDice':s_dice2, 'StanzaHellinger':s_hel2, 'StanzaJaccard':s_jac2, 'StanzaJSdivergence':s_jen2}
Tyndale2 = pd.DataFrame(Tyndale2)

In [63]:
Tyndale_all = pd.concat([Tyndale, Tyndale1, Tyndale2], axis = 0)

In [64]:
all_index = pd.concat([NA28_all,SBL_all,Tyndale_all], axis = 1)

In [65]:
all_index.to_excel('Stanza_index.xlsx',index = False)